In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import logging
import re
from urllib.parse import urljoin

def get_real_pdf_url(viewer_url):
    try:
        # Chrome 옵션 설정
        chrome_options = Options()
        chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")

        # Chrome 드라이버 설정
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        
        # 페이지 로드
        driver.get(viewer_url)
        
        # iframe 요소 찾기
        iframe = driver.find_element(By.ID, 'viewerFrame')
        src = iframe.get_attribute('src')
        
        if not src:
            logging.error("iframe의 src 속성을 찾을 수 없습니다")
            return None
        
        # src URL 절대 경로 변환
        iframe_url = urljoin('https://pcms.samsunglife.com', src)
        logging.info(f"iframe URL: {iframe_url}")
        
        # iframe 페이지로 이동
        driver.get(iframe_url)
        
        # 페이지 소스에서 PDF 경로 찾기
        page_source = driver.page_source
        patterns = [
            r'"filepath"\s*:\s*[\'"](.+?\.pdf)[\'"]',
            r'"downloadURL"\s*:\s*[\'"](.+?\.pdf)[\'"]'
        ]
        
        for pattern in patterns:
            match = re.search(pattern, page_source)
            if match:
                pdf_path = match.group(1)
                pdf_url = urljoin('https://pcms.samsunglife.com', pdf_path)
                logging.info(f"PDF 다운로드 URL 찾음: {pdf_url}")
                return pdf_url
        
        logging.error("PDF 파일 경로를 찾을 수 없음")
        return None
        
    except Exception as e:
        logging.error(f"PDF URL 추출 중 오류 발생: {str(e)}")
        return None
    finally:
        driver.quit()

# 사용 예시
viewer_url = "https://pcms.samsunglife.com/partnerpage/CustomerPage_Unit.jsp?goodsCode=LP0345010&docType=301&saleDate=20240401&pageGubun=prdt"
pdf_url = get_real_pdf_url(viewer_url)
print(pdf_url)

https://pcms.samsunglife.com/uploadDir/doc/2024/0401/LP0345010/301/1711698169038.pdf
